In [1]:
import argparse
from types import SimpleNamespace
import sys
sys.path.append('/data/lige/HKN')# Please change accordingly!
config_args = {
    'training_config': {
        'use_geoopt': (False, "which manifold class to use, if false then use basd.manifold"),
        'lr': (0.001, 'learning rate'),
        'dropout': (0.2, 'dropout probability'),
        'cuda': (0, 'which cuda device to use (-1 for cpu training)'),
        'epochs': (100, 'maximum number of epochs to train for'),
        'weight_decay': (0., 'l2 regularization strength'),
        'optimizer': ('radam', 'which optimizer to use, can be any of [rsgd, radam]'),
        'momentum': (0.999, 'momentum in optimizer'),
        'patience': (150, 'patience for early stopping'),
        'seed': (1234, 'seed for training'),
        'log_freq': (1, 'how often to compute print train/val metrics (in epochs)'),
        'eval_freq': (1, 'how often to compute val metrics (in epochs)'),
        'save': (0, '1 to save model and logs and 0 otherwise'),
        'save_dir': (None, 'path to save training logs and model weights (defaults to logs/task/date/run/)'),
        'sweep_c': (0, ''),
        'lr_reduce_freq': (None, 'reduce lr every lr-reduce-freq or None to keep lr constant'),
        'gamma': (0.5, 'gamma for lr scheduler'),
        'print-epoch': (True, ''),
        'grad-clip': (None, 'max norm for gradient clipping, or None for no gradient clipping'),
        'min-epochs': (100, 'do not early stop before min-epochs')
    },
    'model_config': {
        'task': ('nc', 'which tasks to train on, can be any of [lp, nc]'),
        'model': ('BKNet', 'which encoder to use, can be any of [Shallow, MLP, HNN, GCN, GAT, HyperGCN, HyboNet,BKN]'),
        'dim': (32, 'embedding dimension'),
        'manifold': ('PoincareBall', 'which manifold to use, can be any of [Euclidean, Hyperboloid, PoincareBall, Lorentz]'),
        'c': (1.0, 'hyperbolic radius, set to None for trainable curvature'),
        'r': (2., 'fermi-dirac decoder parameter for lp'),
        't': (1., 'fermi-dirac decoder parameter for lp'),
        'margin': (2., 'margin of MarginLoss'),
        'pretrained_embeddings': (None, 'path to pretrained embeddings (.npy file) for Shallow node classification'),
        'pos_weight': (0, 'whether to upweight positive class in node classification tasks'),
        'num_layers': (2, 'number of hidden layers in encoder'),
        'bias': (1, 'whether to use bias (1) or not (0)'),
        'act': ('relu', 'which activation function to use (or None for no activation)'),
        'n_heads': (4, 'number of attention heads for graph attention networks, must be a divisor dim'),
        'alpha': (0.2, 'alpha for leakyrelu in graph attention networks'),
        'double_precision': ('1', 'whether to use double precision'),
        'use_att': (0, 'whether to use hyperbolic attention or not'),
        'local_agg': (0, 'whether to local tangent space aggregation or not'),
        'kernel_size': (8, 'number of kernels'),
        'KP_extent': (0.66, 'influence radius of each kernel point'),
        'radius': (1, 'radius used for kernel point init'),
        'deformable': (False, 'deformable kernel'),
        'linear_before': (64, 'dim of linear before gcn')
    },
    'data_config': {
        'dataset': ('cornell', 'which dataset to use'),
        'batch_size': (32, 'batch size for gc'),
        'val_prop': (0.05, 'proportion of validation edges for link prediction'),
        'test_prop': (0.1, 'proportion of test edges for link prediction'),
        'use_feats': (1, 'whether to use node features or not'),
        'normalize_feats': (1, 'whether to normalize input node features'),
        'normalize_adj': (1, 'whether to row-normalize the adjacency matrix'),
        'split_seed': (1234, 'seed for data splits (train/test/val)'),
        'split_graph': (False, 'whether to split the graph')
    }
}

# 将所有参数转换为 SimpleNamespace
args = SimpleNamespace(
    **{k: v[0] for config in config_args.values() for k, v in config.items()}
)
args

namespace(use_geoopt=False,
          lr=0.001,
          dropout=0.2,
          cuda=0,
          epochs=100,
          weight_decay=0.0,
          optimizer='radam',
          momentum=0.999,
          patience=150,
          seed=1234,
          log_freq=1,
          eval_freq=1,
          save=0,
          save_dir=None,
          sweep_c=0,
          lr_reduce_freq=None,
          gamma=0.5,
          print-epoch=True,
          grad-clip=None,
          min-epochs=100,
          task='nc',
          model='BKNet',
          dim=32,
          manifold='PoincareBall',
          c=1.0,
          r=2.0,
          t=1.0,
          margin=2.0,
          pretrained_embeddings=None,
          pos_weight=0,
          num_layers=2,
          bias=1,
          act='relu',
          n_heads=4,
          alpha=0.2,
          double_precision='1',
          use_att=0,
          local_agg=0,
          kernel_size=8,
          KP_extent=0.66,
          radius=1,
          deformable=False,
     

In [2]:
from __future__ import division
from __future__ import print_function

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

import datetime
import json
import logging
from optim import RiemannianAdam, RiemannianSGD
import os
import pickle
import time

import numpy as np
import torch
from config import parser
from models.base_models import NCModel, LPModel, GCModel
from utils.data_utils import load_data, get_nei, GCDataset, split_batch
from utils.train_utils import get_dir_name, format_metrics
from utils.eval_utils import acc_f1

from geoopt import ManifoldParameter as geoopt_ManifoldParameter
from manifolds.base import ManifoldParameter as base_ManifoldParameter

/data/lige/miniconda3/envs/HKNet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#choose which manifold class to follow 
if args.use_geoopt == False:
    ManifoldParameter = base_ManifoldParameter
else:
    ManifoldParameter = geoopt_ManifoldParameter
np.random.seed(args.seed)#args.seed
torch.manual_seed(args.seed)#args.seed
if int(args.cuda):#args.double_precision
    torch.set_default_dtype(torch.float64)
if int(args.cuda) >= 0:#args.cuda
    torch.cuda.manual_seed(args.seed)#args.seed
args.device = 'cuda:' + str(args.cuda) if int(args.cuda) >= 0 else 'cpu' #args.device actually,<-args.cuda
# args.device = 'cpu'
args.patience = args.epochs if not args.patience else args.patience #args.patience<-args.epochs|args.patience
"""
logging.getLogger().setLevel(logging.INFO)
if args.save:
    if not args.save_dir:
        dt = datetime.datetime.now()
        date = f"{dt.year}_{dt.month}_{dt.day}"
        models_dir = os.path.join('logs', args.task, date)
        save_dir = get_dir_name(models_dir)
    else:
        save_dir = args.save_dir
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    logging.basicConfig(level=logging.INFO,
                        handlers=[
                            logging.FileHandler(
                                os.path.join(save_dir, 'log.txt')),
                            logging.StreamHandler()
                        ])
"""

print(f'Using: {args.device}')
print("Using seed {}.".format(args.seed))
print(f"Dataset: {args.dataset}")

# Load data
data = load_data(args, os.path.join('data', args.dataset))
if args.task == 'gc':
    args.n_nodes, args.feat_dim = data['features'][0].shape
else:
    args.n_nodes, args.feat_dim = data['features'].shape
if args.task == 'nc':
    Model = NCModel
    args.n_classes = int(data['labels'].max() + 1)
    args.data = data
    print(f'Num classes: {args.n_classes}')
elif args.task == 'gc':
    Model = GCModel
    args.n_classes = int(data['labels'].max() + 1)
    print(f'Num classes: {args.n_classes}')
else:
    args.nb_false_edges = len(data['train_edges_false'])
    args.nb_edges = len(data['train_edges'])
    if args.task == 'lp':
        Model = LPModel
        args.n_classes = 2

if not args.lr_reduce_freq:
    args.lr_reduce_freq = args.epochs



Using: cuda:0
Using seed 1234.
Dataset: cornell
Num classes: 5


/data/lige/HKN/utils/data_utils.py:300: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G, sorted(G.nodes()))


In [5]:
####################################Checks####################################
print(data.keys())
#Note it's always stored in sparse matrix
print(data['adj_train'].todense().shape)
print(data['features'].shape)
data['features'][]
####################################Checks####################################

dict_keys(['adj_train', 'features', 'labels', 'idx_train', 'idx_val', 'idx_test', 'adj_train_norm'])
(183, 183)
torch.Size([183, 1703])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [5]:
# Model and optimizer
model = Model(args)
print(str(model))
no_decay = ['bias', 'scale']
optimizer_grouped_parameters = [{
    'params': [
        p for n, p in model.named_parameters()
        if p.requires_grad and not any(
            nd in n
            for nd in no_decay) and not isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    args.weight_decay
}, {
    'params': [
        p for n, p in model.named_parameters() if p.requires_grad and any(
            nd in n
            for nd in no_decay) or isinstance(p, ManifoldParameter)
    ],
    'weight_decay':
    0.0
}]
if args.optimizer == 'radam':
    optimizer = RiemannianAdam(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
elif args.optimizer == 'rsgd':
    optimizer = RiemannianSGD(params=optimizer_grouped_parameters,
                                lr=args.lr,
                                stabilize=10)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=int(
                                                args.lr_reduce_freq),
                                                gamma=float(args.gamma))
tot_params = sum([np.prod(p.size()) for p in model.parameters()])
model = model.to(args.device)
for x, val in data.items():
    if torch.is_tensor(data[x]):
        data[x] = data[x].to(args.device)
print(f"Total number of parameters: {tot_params}")


NCModel(
  (encoder): BKNet(
    (linear_before): BLinear(in_features=1703, out_features=64, c=tensor([1.], device='cuda:0'))
    (layers): Sequential(
      (0): KPGraphConvolution(
        (net): KernelPointAggregation(
          (linears): ModuleList(
            (0): BLayer(
              (linear): BLinear(in_features=64, out_features=32, c=1)
              (act): BAct(c_in=tensor([1.], device='cuda:0'), c_out=tensor([1.], device='cuda:0'))
            )
            (1): BLayer(
              (linear): BLinear(in_features=64, out_features=32, c=1)
              (act): BAct(c_in=tensor([1.], device='cuda:0'), c_out=tensor([1.], device='cuda:0'))
            )
            (2): BLayer(
              (linear): BLinear(in_features=64, out_features=32, c=1)
              (act): BAct(c_in=tensor([1.], device='cuda:0'), c_out=tensor([1.], device='cuda:0'))
            )
            (3): BLayer(
              (linear): BLinear(in_features=64, out_features=32, c=1)
              (act): BAct(

In [6]:
# Train model for nc:
t_total = time.time()
counter = 0
best_val_metrics = model.init_metric_dict()
best_test_metrics = None
best_emb = None
if args.n_classes > 2:
    f1_average = 'micro'
else:
    f1_average = 'binary'

best_val_metrics

{'acc': -1, 'f1': -1}

In [7]:
#Here we go
if args.model == 'HKPNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device)
elif args.model == 'BKNet':
    nei, nei_mask = get_nei(data['adj_train'])
    nei = nei.to(args.device)
    nei_mask = nei_mask.to(args.device) #nei/nei_mask on cuda now

nei

tensor([[ 0,  0,  0,  ...,  0,  0,  0],
        [90,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [57,  0,  0,  ...,  0,  0,  0]], device='cuda:0')

In [8]:
####################################Checks####################################
import manifolds
import models.encoders as encoders
import layers.B_layers as B_layers
manifold=getattr(manifolds, 'PoincareBall')()
print(manifold)
BKNet=getattr(encoders, 'BKNet')(torch.tensor([args.c]).to(args.device),args)
#print(BKNet)
dims, acts, curvatures = B_layers.get_dim_act_curv(args)
print(dims,'\n',acts,'\n',BKNet.curvatures)
BKNet.curvatures.append(torch.tensor([args.c]).to(args.device))
print(BKNet.curvatures)
####################################Checks####################################


[1703, 32, 32] 
 [<function relu at 0x7f534890b2e0>, <function relu at 0x7f534890b2e0>] 
 [tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0')]
[tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0'), tensor([1.], device='cuda:0')]


In [9]:
model.encode(data['features'], (nei, nei_mask))

TypeError: 'NoneType' object is not callable

In [ ]:

#Here we go
        for epoch in range(args.epochs):
            t = time.time()
            model.train()
            optimizer.zero_grad()
            if args.model == 'HKPNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))
                # print(embeddings.isnan().sum())
            elif args.model == 'BKNet':
                embeddings = model.encode(data['features'], (nei, nei_mask))#if correctly, embeddings on cuda as well
            else:
                embeddings = model.encode(data['features'], data['adj_train_norm'])
            train_metrics = model.compute_metrics(embeddings, data, 'train')
            train_metrics['loss'].backward()
            if args.grad_clip is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
            optimizer.step()
            lr_scheduler.step()
            if (epoch + 1) % args.log_freq == 0:
                logging.info(" ".join([
                    'Epoch: {:04d}'.format(epoch + 1),
                    'lr: {}'.format(lr_scheduler.get_last_lr()),
                    format_metrics(train_metrics, 'train'),
                    'time: {:.4f}s'.format(time.time() - t)
                ]))
            with torch.no_grad():
                if (epoch + 1) % args.eval_freq == 0:
                    model.eval()
                    if args.model == 'HKPNet':
                        embeddings = model.encode(data['features'], (nei, nei_mask))
                    elif args.model == 'BKNet':
                        embeddings = model.encode(data['features'], (nei, nei_mask))
                    else:
                        embeddings = model.encode(data['features'],
                                                data['adj_train_norm'])
                    val_metrics = model.compute_metrics(embeddings, data, 'val')
                    if (epoch + 1) % args.log_freq == 0:
                        logging.info(" ".join([
                            'Epoch: {:04d}'.format(epoch + 1),
                            format_metrics(val_metrics, 'val')
                        ]))
                    if model.has_improved(best_val_metrics, val_metrics):
                        best_test_metrics = model.compute_metrics(
                            embeddings, data, 'test')
                        best_emb = embeddings.cpu()
                        if args.save:
                            np.save(os.path.join(save_dir, 'embeddings.npy'),
                                    best_emb.detach().numpy())
                        best_val_metrics = val_metrics
                        counter = 0
                    else:
                        counter += 1
                        if counter == args.patience and epoch > args.min_epochs:
                            logging.info("Early stopping")
                            break

        logging.info("Optimization Finished!")
        logging.info("Total time elapsed: {:.4f}s".format(time.time() - t_total))
        if not best_test_metrics:
            model.eval()
            best_emb = model.encode(data['features'], data['adj_train_norm'])
            best_test_metrics = model.compute_metrics(best_emb, data, 'test')
        logging.info(" ".join(
            ["Val set results:",
            format_metrics(best_val_metrics, 'val')]))
        logging.info(" ".join(
            ["Test set results:",
            format_metrics(best_test_metrics, 'test')]))
        if args.save:
            np.save(os.path.join(save_dir, 'embeddings.npy'),
                    best_emb.cpu().detach().numpy())
            if hasattr(model.encoder, 'att_adj'):
                filename = os.path.join(save_dir, args.dataset + '_att_adj.p')
                pickle.dump(model.encoder.att_adj.cpu().to_dense(),
                            open(filename, 'wb'))
                print('Dumped attention adj: ' + filename)

            torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))
            json.dump(vars(args), open(os.path.join(save_dir, 'config.json'), 'w'))
            logging.info(f"Saved model in {save_dir}")